In [334]:
import os
import time
import torch
import torch as tc
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import matplotlib.pyplot as plt
#get_ipython().magic('matplotlib inline')
from PIL import Image
import numpy as np
import sys,os
import math
import gc
import torch.nn.functional as F


In [146]:
import torch.backends.cudnn as cudnn
import torch
import sys
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION')
#from subprocess import call
#call(["nvcc", "--version"])
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
#call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
print('Active CUDA Device: GPU', torch.cuda.current_device())

__Python VERSION: 3.5.3 |Anaconda 4.4.0 (64-bit)| (default, Mar  6 2017, 11:58:13) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
__pyTorch VERSION: 0.2.0_4
__CUDA VERSION
__CUDNN VERSION: 6021
__Number CUDA Devices: 1
__Devices
Active CUDA Device: GPU 0


In [147]:
def save_images(netG, noise, outputDir,epoch):
   # the first 64 samples from the mini-batch are saved.
   fake,_ = netG(fixed_noise)
   vutils.save_image(fake.data[0:64,:,:,:],
		   '%s/fake_samples_epoch_%03d.png' % (outputDir, epoch), nrow=8)

def save_models(netG, netD, outputDir, epoch):
   torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (outputDir, epoch))
   torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (outputDir, epoch))


In [767]:
class _netD_1(nn.Module):
    def __init__(self, ngpu, nz, nc, ndf,  n_extra_layers_d, n_classes):
        super(_netD_1, self).__init__()
        self.ngpu = ngpu
        self.conv1 = nn.Conv2d(in_channels = nc, out_channels = ndf, kernel_size=4, stride=2, padding=1, bias=False)
        self.conv2 = nn.Conv2d(in_channels = ndf, out_channels = ndf*2, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch2 = nn.BatchNorm2d(ndf * 2)
        self.conv3 = nn.Conv2d(in_channels = ndf*2, out_channels = ndf*4, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch3 = nn.BatchNorm2d(ndf * 4)
        self.conv4 = nn.Conv2d(in_channels = ndf*4, out_channels = ndf*8, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch4 = nn.BatchNorm2d(ndf * 8)
        
        self.final_conv = nn.Conv2d(in_channels=ndf*8, out_channels=n_classes,kernel_size=2,stride=1,padding=0,bias=False)
        
    def forward(self, x):
        print('0',x.size())

        x = F.leaky_relu(self.conv1(x),0.02,inplace=True)
        print('1',x.size())
        x = F.leaky_relu(self.batch2(self.conv2(x)),0.02,inplace=True)
        print('2',x.size())
        x = F.leaky_relu(self.batch3(self.conv3(x)),0.02,inplace=True)
        print('3',x.size())
        x = F.leaky_relu(self.batch4(self.conv4(x)),0.02,inplace=True)   
        print('antes final',x.size())
        x = self.final_conv(x)
        print('depois final',x.size())
        
        x = F.sigmoid(x)

        return(x)
        
  

In [148]:
cudnn.benchmark = True

use_gpu = torch.cuda.is_available()
#use_gpu = False
if use_gpu:
	print("You are using CUDA. If it is not what you want, manually set this as False!")

You are using CUDA. If it is not what you want, manually set this as False!


nc = 3
ngpu = 1
nz = 100
ngf = 64
ndf = 64
n_extra_d = 0
n_extra_g = 1 # Aqui a jogada é que o gerador deve ser mais poderoso q o detetive
imageSize = 64


In [149]:
# This should, in the future, be set in CLI
chosen_dataset = 'CIFAR10'

datasets = {
    'MNIST': torchvision.datasets.MNIST,
    'CIFAR10': torchvision.datasets.CIFAR10,
    'ANIME': '/home/gabriel/Redes Neurais/Projeto_Final_GANS/Tutorial_2/dataset/min_anime-faces',
    #'FashionMNIST': torchvision.datasets.FashionMNIST
}

dataset = datasets[chosen_dataset]

In [150]:
print(dataset)

<class 'torchvision.datasets.cifar.CIFAR10'>


In [151]:
possible_parameters = {
    'MNIST': {
        'ndf': 64,
        'ngf': 64,
        'nz': 50,
        'nc': 1,
        'n_classes' : 10,
    },
    'CIFAR10': {
        'ndf': 32,
        'ngf': 32,
        'nz': 100,
        'nc': 3,
        'n_extra_d' : 0,
        'n_extra_g' : 1, # Aqui a jogada é que o gerador deve ser mais poderoso q o detetive
        'imageSize' : 32,
        'n_classes' : 10,
        'ngpu' : 1,		
    },
    'ANIME': {
        'nc' : 3,
        'ngpu' : 1,
        'nz' : 100,
        'ngf' : 64,
        'ndf' : 64,
        'n_extra_d' : 0,
        'n_extra_g' : 0, # Aqui a jogada é que o gerador deve ser mais poderoso q o detetive
        'imageSize' : 64,
        'n_classes' : 1


    }
    #'FashionMNIST': {}
}

In [232]:
ngf = possible_parameters[chosen_dataset]['ngf']
ndf = possible_parameters[chosen_dataset]['ndf']
nz = possible_parameters[chosen_dataset]['nz']
nc = possible_parameters[chosen_dataset]['nc']
imageSize = possible_parameters[chosen_dataset]['imageSize']
n_classes = possible_parameters[chosen_dataset]['n_classes']
ngpu = possible_parameters[chosen_dataset]['ngpu']
n_extra_d = possible_parameters[chosen_dataset]['n_extra_d']
n_extra_g = possible_parameters[chosen_dataset]['n_extra_g']

## Creating the Dataset!

In [233]:
dataset_done = dataset('./datasets', train=True, download=True, transform=transform)

Files already downloaded and verified


In [234]:
batch_size=64


In [592]:
if dataset == 'ANIME':
    dataset = dset.ImageFolder(
    root='/home/gabriel/Redes Neurais/Projeto_Final_GANS/Tutorial_2/dataset/min_anime-faces',
    transform=transforms.Compose([
            transforms.Scale((imageSize, imageSize)),
            # transforms.CenterCrop(opt.imageSize),
            transforms.ToTensor(),
            #transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)), # bring images to (-1,1)
        ])
    )
else:
    transform=transforms.Compose([
                transforms.Scale((imageSize, imageSize)),
                transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)), # bring images to (-1,1)
            ]) 
    dataset_done = dataset('./datasets', train=True, download=True, transform=transform)
    dataloader = tc.utils.data.DataLoader(dataset_done, batch_size=batch_size, shuffle=True, num_workers=1
                                          )


Files already downloaded and verified


## Criando os modelos

In [768]:
class _netD_1(nn.Module):
    def __init__(self, ngpu, nz, nc, ndf,  n_extra_layers_d, n_classes):
        super(_netD_1, self).__init__()
        self.ngpu = ngpu
        self.conv1 = nn.Conv2d(in_channels = nc, out_channels = ndf, kernel_size=4, stride=2, padding=1, bias=False)
        self.conv2 = nn.Conv2d(in_channels = ndf, out_channels = ndf*2, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch2 = nn.BatchNorm2d(ndf * 2)
        self.conv3 = nn.Conv2d(in_channels = ndf*2, out_channels = ndf*4, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch3 = nn.BatchNorm2d(ndf * 4)
        self.conv4 = nn.Conv2d(in_channels = ndf*4, out_channels = ndf*8, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch4 = nn.BatchNorm2d(ndf * 8)
        
        self.final_conv = nn.Conv2d(in_channels=ndf*8, out_channels=n_classes,kernel_size=2,stride=1,padding=0,bias=False)
        
    def forward(self, x):
        #print('0',x.size())

        x = F.leaky_relu(self.conv1(x),0.02,inplace=True)
        #print('1',x.size())
        x = F.leaky_relu(self.batch2(self.conv2(x)),0.02,inplace=True)
        #print('2',x.size())
        x = F.leaky_relu(self.batch3(self.conv3(x)),0.02,inplace=True)
        #print('3',x.size())
        x = F.leaky_relu(self.batch4(self.conv4(x)),0.02,inplace=True)   
        #print('antes final',x.size())
        x = self.final_conv(x)
        #print('depois final',x.size())
        
        x = F.sigmoid(x)

        return(x)
        
  

In [792]:
class _netG_1(nn.Module):
    def __init__(self, ngpu, nz, nc , ngf, n_extra_layers_g):
        super(_netG_1, self).__init__()
        self.ngpu = ngpu
        
        self.convt1 = nn.ConvTranspose2d(in_channels = nz, out_channels = ngf * 8, kernel_size=4, stride=2, padding=0, bias=False)
        self.batch1 = nn.BatchNorm2d(ngf*8)
        self.convt2 = nn.ConvTranspose2d(in_channels = ngf * 8, out_channels = ngf * 4, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch2 = nn.BatchNorm2d(ngf*4)
        self.convt3 = nn.ConvTranspose2d(in_channels = ngf * 4, out_channels = ngf * 2, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch3 = nn.BatchNorm2d(ngf*2)
        self.convt4 = nn.ConvTranspose2d(in_channels = ngf*2, out_channels = ngf, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch4 = nn.BatchNorm2d(ngf)
        
        self.final_conv = nn.ConvTranspose2d(in_channels=ngf, out_channels=nc,kernel_size=1,stride=1,padding=0,bias=False)
    def forward(self, x):
        x = F.leaky_relu(self.batch1(self.convt1(x)),0.02,inplace=True)
        x = F.leaky_relu(self.batch2(self.convt2(x)),0.02,inplace=True)
        x = F.leaky_relu(self.batch3(self.convt3(x)),0.02,inplace=True)
        x = F.leaky_relu(self.batch4(self.convt4(x)),0.02,inplace=True)
        x = self.final_conv(x)
        x = F.tanh(x)
        
        return (x)

In [793]:
print(ngpu, nz, nc, ngf, n_extra_g)
netG = _netG_1(ngpu, nz, nc, ngf, n_extra_g)
#netG_parallel = torch.nn.DataParallel(_netG_1(ngpu, nz, nc, ngf, n_extra_g))
print(ngpu, nz, nc, ndf, n_extra_d,n_classes)
netD = _netD_1(ngpu, nz, nc, ndf, n_extra_d,n_classes)
#netD_parallel = torch.nn.DataParallel(_netD_1(ngpu, nz, nc, ndf, n_extra_d))

1 100 3 32 1
1 100 3 32 0 10


In [794]:
print(netD)

_netD_1 (
  (conv1): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (conv2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (batch2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (conv3): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (batch3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
  (conv4): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (batch4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
  (final_conv): Conv2d(256, 10, kernel_size=(2, 2), stride=(1, 1), bias=False)
)


## Inicializador de pesos

In [795]:
def weights_init(m):
	classname = m.__class__.__name__
	if classname.find('Conv') != -1:
		m.weight.data.normal_(0.0, 0.02)
	elif classname.find('BatchNorm') != -1:
		m.weight.data.normal_(1.0, 0.02)
		m.bias.data.fill_(0)

In [796]:
netG.apply(weights_init)
#netG_parallel.apply(weights_init)
netD.apply(weights_init)
#netD_parallel.apply(weights_init)
print()

## Losses

In [797]:
criterion = nn.BCELoss()
criterion_MSE = nn.MSELoss()

## Sizes of the tensors

In [798]:
input = torch.FloatTensor(batch_size, 3, imageSize, imageSize)
print(input.size())
noise = torch.FloatTensor(batch_size, nz, 1, 1)
print(noise.size())

torch.Size([64, 3, 32, 32])
torch.Size([64, 100, 1, 1])


In [799]:
binary=False
#Ele testa pergunta se vc quer que o seu Z venha da distribuição bernoulli
if binary:
	bernoulli_prob = torch.FloatTensor(batch_size, nz, 1, 1).fill_(0.5)
	fixed_noise = torch.bernoulli(bernoulli_prob)
else:
	fixed_noise = torch.FloatTensor(batch_size, nz, 1, 1).normal_(0, 1)

In [800]:
label = torch.FloatTensor(batch_size,n_classes)
print(label.size())
real_label = 1
fake_label = 0

torch.Size([64, 10])


In [801]:
one_hot = torch.LongTensor(64, 10).zero_()


## Broadcast then to the GPU

In [802]:
if use_gpu:
	netD.cuda()
	netG.cuda()
	#netD_parallel.cuda()
	#netG_parallel.cuda()
	criterion = criterion.cuda()
	criterion_MSE = criterion_MSE.cuda()
	input,label = input.cuda(), label.cuda()
	noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

## Turn then on Variables

In [803]:
input = Variable(input)
label = Variable(label)
noise = Variable(noise)
fixed_noise = Variable(fixed_noise)

In [804]:
one_hot = Variable(one_hot)

In [805]:
print(type(label))
print(label.size())

<class 'torch.autograd.variable.Variable'>
torch.Size([64, 10])


## Parâmetros de treinamento

In [806]:
beta1, beta2 = 0.9,0.999
lr = 2.0e-4
optimizerD = optim.Adam(netD.parameters(), lr = lr, betas = (beta1, beta2))
#optimizerD = optim.Adam(netD_parallel.parameters(), lr = lr, betas = (beta1, beta2))

optimizerG = optim.Adam(netG.parameters(), lr = lr, betas = (beta1, beta2))
#optimizerG = optim.Adam(netG_parallel.parameters(), lr = lr, betas = (beta1, beta2))


## Output_dir

In [807]:

outputDir = 'outputdir_train_classifier'

try:
	os.makedirs(outputDir)
except OSError as err:
	print("OS error: {0}".format(err))

OS error: [Errno 17] File exists: 'outputdir_train_classifier'


In [808]:
print(type(label))

<class 'torch.autograd.variable.Variable'>


In [809]:
a = tc.randn(64,3,32,32)
a = Variable(a)
output = netD(a.cuda())
print(output.size())

torch.Size([64, 10, 1, 1])


In [810]:
b = tc.randn(64,100,1,1)
b = Variable(b)
output = netG(b.cuda())
print(output.size())

torch.Size([64, 3, 32, 32])



## Treinamento 

In [813]:
def train_our(num_epochs, dataloader, netD, netG, d_labelSmooth, outputDir,
						model_option =1,binary = False, epoch_interval = 100,
						D_steps = 1, G_steps = 1):
	use_gpu = tc.cuda.is_available()
	print('Lets train!')
	#if (batch_size/D_steps is not 0):
		#raise ValueError('Use batch_size multiple of D_steps')
	for epoch in range(num_epochs):
		start_iter = time.time()  
		D_x = 0
		D_G_z1 = 0
		D_G_z2 = 0
		errD_acum = 0
		errG_acum = 0
		

		for j, data in enumerate(dataloader, 0):

			for z in range(D_steps):
				if z > 3:
					raise ValueError('KEEP IT LOW!')
			
				############################
				# (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
				# 1A - Train the detective network in the Real Dataset
				###########################
				# train with real
				netD.zero_grad()
				one_hot.data.fill_(0)
				#real_cpu, _ = data
				#print('ITALOS',data[0].size()[0])
				start = z*(int(data[0].size()[0]/D_steps))
				end = (z+1)*int(data[0].size()[0]/D_steps)
				#real_cpu = data[0][z*(int(data[0].size()[0]/D_steps)):(z+1)*int(data[0].size()[0]/D_steps)]				
				inp, target = data
				target_ = tc.unsqueeze(target,1)
				one_hot.scatter_(1, target_, 1)
				real_cpu = data[0][start:end]
				if (epoch == 0 and z == 0 ):
					vutils.save_image(real_cpu[0:64,:,:,:],
					'%s/real_samples.png' % outputDir, nrow=8)
				
				batch_size = real_cpu.size(0)
				input.data.resize_(real_cpu.size()).copy_(real_cpu)
				label.data.resize_(batch_size,10).copy_(one_hot.data) # use smooth label for discriminator
				output = netD(input)
				errD_real = criterion(output, label)
				errD_real.backward()

				#######################################################
				#######################################################
				# 1B - Train the detective network in the False Dataset
				#######################################################

				D_x += output.data.mean()
				# train with fake
				noise.data.resize_(batch_size, nz, 1, 1)
				if binary:
					bernoulli_prob.resize_(noise.data.size())
					noise.data.copy_(2*(torch.bernoulli(bernoulli_prob)-0.5))
				else:
					noise.data.normal_(0, 1)
				fake = netG(noise)
#				fake,z_prediction = netG(noise)
				label.data.fill_(fake_label)
				output = netD(fake.detach()) # add ".detach()" to avoid backprop through G
				errD_fake = criterion(output, label)
				errD_fake.backward() # gradients for fake/real will be accumulated
				# ERROR MEAN
				D_G_z1 += output.data.mean()
				
				errD_acum += errD_real.data[0] + errD_fake.data[0]
				
				optimizerD.step() # .step() can be called once the gradients are computed

				#######################################################
			
				# PARADA PARA VER O Q ESTÁ ACONTENDO
		
			for a in range(G_steps):
				#print('interacao = ',a, 'de ',G_steps )
				# G_steps > D_steps (G_steps \geq D_steps)
				if a > 3:
					raise ValueError('KEEP IT LOW!')
		
				#######################################################
				# (2) Update G network: maximize log(D(G(z)))
				#  Train the faker with de output from the Detective (but don't train the Detective)
				#############3#########################################
				netG.zero_grad()
				label.data.fill_(real_label) # fake labels are real for generator cost
				output = netD(fake)
				errG = criterion(output, label)
				errG.backward(retain_graph = True) # True if backward through the graph for the second time
				#errG.backward() # True if backward through the graph for the second time
				
				if model_option == 2: # with z predictor
					errG_z = criterion_MSE(z_prediction, noise)
					errG_z.backward()
				D_G_z2 += output.data.mean()
				errG_acum += errG.data[0]
				#D_G_z2 = output.data.mean()
				#errG_acum = errG				
				optimizerG.step()

		print('epoch = ',epoch)
		
		end_iter = time.time()        
		#Print the info
		print('[%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f Elapsed %.2f s'
			  % (epoch, num_epochs, errD_acum/D_steps, errG_acum/G_steps, D_x, D_G_z1, D_G_z2, end_iter-start_iter))

		#Save a grid with the pictures from the dataset, up until 64
		save_images(netG = netG, noise = fixed_noise, outputDir = outputDir, epoch = epoch)
		   
		if epoch % epoch_interval == 0:
			# do checkpointing
			save_models(netG = netG, netD = netD, outputDir = outputDir, epoch = epoch)


In [814]:

num_epochs = 100
d_labelSmooth = 0.2

train_our(num_epochs, dataloader, netD,netG,d_labelSmooth, outputDir)

Lets train!


/home/gabriel/anaconda3/envs/py35/lib/python3.5/site-packages/torch/nn/functional.py:767: UserWarning: Using a target size (torch.Size([64, 10])) that is different to the input size (torch.Size([64, 10, 1, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


RuntimeError: Expected tensor [64 x 10] and index [16 x 1] to have the same size in dimension 1 at /opt/conda/conda-bld/pytorch_1503968623488/work/torch/lib/TH/generic/THTensorMath.c:518